In [3]:
import numpy as np
import scipy.special

In [39]:
class NerualNetwork:
    
    def __init__(self,inputNodes,hiddenNodes,outputNodes,learningRate):
        self.inodes = inputNodes
        self.hnodes = hiddenNodes
        self.onodes = outputNodes
        
        # activation function is the sigmoid function
     
        self.activation_function = lambda x: scipy.special.expit(x)
        
        self.ihw = np.random.normal(0.0, pow(self.hnodes, -0.5), (self.hnodes, self.inodes)) #(numpy.random.rand(self.hnodes, self.inodes) - 0.5)
        self.how = np.random.normal(0.0, pow(self.onodes, -0.5), (self.onodes, self.hnodes)) #(numpy.random.rand(self.onodes, self.hnodes) - 0.5) 
        self.lr = learningRate
        

    def train(self,input_list,target_list):
        # convert inputs list to 2d array
        inputs = np.array(inputs_list, ndmin=2).T
        targets = np.array(target_list, ndmin=2).T
        
        print(inputs)
        # calculate signals into hidden layer
        hidden_inputs = np.dot(self.ihw, inputs)
        # calculate the signals emerging from hidden layer
        hidden_outputs = self.activation_function(hidden_inputs)

        # calculate signals into final output layer
        final_inputs = np.dot(self.how, hidden_outputs)
        # calculate the signals emerging from final output layer
        final_outputs = self.activation_function(final_inputs)

        output_errors = targets - final_outputs

    def query(self, inputs_list):
        # convert inputs list to 2d array
        inputs = np.array(inputs_list, ndmin=2).T
        
        print(inputs)
        # calculate signals into hidden layer
        hidden_inputs = np.dot(self.ihw, inputs)
        # calculate the signals emerging from hidden layer
        hidden_outputs = self.activation_function(hidden_inputs)

        # calculate signals into final output layer
        final_inputs = np.dot(self.how, hidden_outputs)
        # calculate the signals emerging from final output layer
        final_outputs = self.activation_function(final_inputs)

        return final_outputs

In [40]:
inputNodes = 3
hiddenNodes = 3
outputNodes = 3

learningRate = 0.3

NN = NerualNetwork(inputNodes,hiddenNodes,outputNodes,learningRate)

In [41]:
print("Input Nodes: {} , Hidden Nodes: {}, Output Nodes: {} , Learning Rate: {} \n".format(NN.inodes,NN.hnodes,NN.onodes,NN.lr))
print("Input to Hidden Connection Matrix: \n {} \n\n Hidden to Output Connection Matrix: \n {}".format(NN.ihw,NN.how))

Input Nodes: 3 , Hidden Nodes: 3, Output Nodes: 3 , Learning Rate: 0.3 

Input to Hidden Connection Matrix: 
 [[ 0.27895618 -0.53812878  0.47784301]
 [-0.0913017  -0.06247916 -0.48192431]
 [ 0.39333516  0.45458831  0.33513022]] 

 Hidden to Output Connection Matrix: 
 [[-0.13664651 -0.19263763  0.57985524]
 [ 0.83857894  0.07944624 -0.42860503]
 [ 0.61396949  0.72237449  0.14270593]]


In [42]:
NN.query([1,2,3])

[[1]
 [2]
 [3]]


array([[0.60044475],
       [0.54269301],
       [0.65621703]])